In [1]:
# imports
import os
import numpy as np
import json
import pandas as pd
import time

In [2]:
# scan text for predefined terms

text = 'We use CNN for anomaly detection. As Convolutional Neural Networks are great for ML.'

def text_match(text):
    ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
    ml_slugs = ml_terms['Slug'].tolist()
    ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
    ml_terms = ml_terms['Term'].tolist()
    #print(ml_slugs)

    contained_slugs = [x for x in ml_slugs if x in text]
    contained_terms = [x for x in ml_terms if x.lower() in text.lower()]
    
    #if 'CNN' or 'ANN' or 'RNN' in contained_slugs:
    #    contained_slugs.remove('NN')
    
    return contained_slugs + contained_terms

# test function
matches = text_match(text)
print(matches)

['CNN', 'ML', 'NN', 'anomaly', 'anomaly detection', 'convolutional neural network', 'detect', 'neural network']


In [3]:
# improved version (full text scan)
# scan text for predefined terms

text = 'We use LSTM for anomaly and object detection. As Convolutional Neural Networks are great for ML.'

pd_ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
ml_terms = pd_ml_terms['Term'].tolist()
ml_slugs = pd_ml_terms['Slug'].tolist()
ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
ml_tags = pd_ml_terms['Tag'].tolist()
ml_tags = [x for x in ml_tags if str(x) != 'nan']

#print(ml_tags)

ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
ml_libs = ml_libs['Python Package'].tolist()

def match_text(haystack, needles, toLower = False, unique = True):
    
    if toLower == True:
        haystack = haystack.lower()
        needles = [x.lower() for x in needles]
    
    if unique == True:
        matches = {x for x in needles if x in haystack}
        matches = list(matches)
    else:
        matches = [x for x in needles if x in haystack]
    
    return matches

def match_tags(haystack):
    df = pd.read_csv('../data/patterns/ml_terms.csv')
    tags = []
    
    df.set_index('Term', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(tag)
        except:
            pass
        
    df.set_index('Slug', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(str(tag))
        except:
            pass
        
    #if 'ANN' in tags or 'CNN' in tags or 'RNN' in tags:
    #    tags.remove('NN')
    
    return list(tags)

#ml_slugs, ml_terms, ml_libs, match_text(haystack, needles, toLower = False, unique = True)
needles = {
    'ml_slugs': ml_slugs,
    'ml_terms': ml_terms,
    'ml_libs': ml_libs,
}
needles_need_str_lower = {
    'ml_slugs': False,
    'ml_terms': True,
    'ml_libs': False,
}

matches = []

matches.extend(match_text(text, ml_terms, True))
matches.extend(match_text(text, ml_slugs, False))
print('matches', matches)

tags = match_tags(matches)
print('tags', tags)

matches ['convolutional neural network', 'object detection', 'detect', 'lstm', 'anomaly', 'neural network', 'ML']
tags ['CNN', 'Object Detection', 'LSTM', 'NN', 'ML']


In [4]:
# python code to package parser

fp1 = '../data/repositories/git/crowdAI/train-schedule-optimisation-challenge-starter-kit/utils/route_penalty.py'
fp2 = '../data/repositories/git/altosaar/food2vec/src/process_scraped_data.py'

def analyze_pythoncode(code):
    meta = {}
    debug = False
    
    ''' version 1
    #print(code)
    code = code.split('\n')
    
    libs = []
    keywords = []
    
    for line in code:
        line = line.strip()
        
        # get libraries
        words = line.split(' ')
        if words[0] == 'import' or words[0] == 'from':
            if debug: print('raw: ' + line.strip())
            lib = line.split(' ')
            if len(lib) > 1:
                lib = lib[1].split('.')[0].strip()
            else:
                break
            if ',' in lib:
                lib = [elem.strip() for elem in lib.split(',')]
                libs.extend(lib)
                if debug: print('parsed: ['+', '.join(lib) + ']')
            else:
                libs.append(lib)
                if debug: print('parsed: '+lib)
                    
        # analyze comments
        if len(line) > 0 and line[0] == '#':
            matches = text_match(line)
            keywords.extend(matches)
            if debug: print('keywords: '+keywords)

    # make values unique
    libs = np.unique(libs).tolist()
    meta['libs'] = libs
    
    keywords = np.unique(keywords).tolist()
    meta['keywords'] = keywords
    '''
    
    # version 2 (full text scan)
    meta = {}
    
    for key, value in needles.items():
        meta[key] = match_text(code, value, needles_need_str_lower[key])
        
    meta['ml_tags'] = match_tags(meta['ml_terms'] + meta['ml_slugs'])
    
    return meta

def analyze_py(filepath):
    meta = {}
    
    with open(filepath, encoding='utf-8', errors="ignore") as fp:
        content = fp.read()
        
    return analyze_pythoncode(content)
    #print(libs)

# test function
meta = analyze_py(fp2)
print(meta)

{'ml_slugs': [], 'ml_terms': ['fit', 'filter', 'train'], 'ml_libs': [], 'ml_tags': []}


In [19]:
# Jupyter Notebook to python code parser

fp1 = '../data/repositories/git/Lemma1/DPFE/DPFE-v0.1.ipynb'
fp2 = '../data/repositories/git/A7med01/Deep-learning-for-Animal-Identification/Identify the Animal_colab.ipynb'
fp3 = '../data/repositories/git/abhiagar90/power_networks/djnagowork/tempsite/src/accumulate_todb/AllPartiesTODB.ipynb'
fp4 = '../data/repositories/git/everAspiring/RegressionAnalysis/RestaurantProfits.ipynb'

def analyze_ipynb(filepath):
    meta = {}
    py = ''
    debug = True
    cells = None
    
    with open(filepath, encoding='utf-8', errors="ignore") as fp:
        c = fp.read()
        try:
            c = json.loads(c)
            cells = c.get('cells')
        except:
            print('no json data')
        
        #if cells == None:
        #    worksheets = c.get('worksheets')
        #    cells = worksheets[0].get('cells')

        #print(cells[0])
            
    if not cells == None:
        for cell in cells:
            if cell.get('source'):
                for source in cell['source']:
                    #print(source.strip())
                    py += str(source)
                py += '\n\n'

            if cell.get('input'):
                for source in cell['input']:
                    #print(source.strip())
                    py += str(source)
                py += '\n\n'

        #print(py)
    return analyze_pythoncode(py)

# test function
meta = analyze_ipynb(fp1)
print(meta)

{'ml_slugs': [], 'ml_terms': ['filter'], 'ml_libs': [], 'ml_tags': []}


In [6]:
# python package pattern matching with limited set

def package_match_simple(libs):
    ml_libs = ['tensorflow','keras','pytorch','sklearn','nltk','mllib','theano','mxnet','lightgbm','scipy','caffe','cntk','auto_ml','OpenNN','h2o']
    matches = set(libs).intersection(ml_libs)
    if not bool(matches):
        matches = {}
    return matches

# test function
matches = package_match_simple(meta['ml_libs'])
print(matches)

{}


In [7]:
# python package pattern matching with extended set

def package_match(libs):
    ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
    ml_packages = ml_libs['Python Package'].tolist()

    matches = set(libs).intersection(ml_packages)
    if not bool(matches):
        matches = {}
    return matches

# test function
matches = package_match(meta['ml_libs'])
print(matches)

{}


In [8]:
# directory walker / run analyze-parser / store results

dir_base = '../data/repositories/git/'
dir1 = 'finnqiao\\cohort_online_retail\\'
dir2 = 'arvindkarir\\retail\\'
dir3 = 'crowdAI\\train-schedule-optimisation-challenge-starter-kit\\'
dir4 = 'Lemma1\\DPFE\\'
dir5 = 'LeadingIndiaAI\\Forest-Fire-Detection-through-UAV-imagery-using-CNNs\\'
dir6 = 'firmai\\financial-machine-learning\\'

# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
                
def dir_walk(dir, store=True):
    meta = []
    dir_base = '../data/repositories/git/'
    path_meta = '/.meta\\git_analyzer_results_v02.json'
    
    for root, dirs, files in os.walk(dir):
        #path = root.split(os.sep)
        #print((len(path) - 1) * '-', os.path.basename(root))
        
        for file in files:
            #print(len(path) * '-', file)
            file_meta = {}
            
            if '.ipynb' in file:
                print('ipynb: '+root+'/'+file)
                file_meta = analyze_ipynb(os.path.join(root, file))
                
            if '.py' in file:
                print('py: '+root+'/'+file)
                file_meta = analyze_py(os.path.join(root, file))
                
            if '.md' in file:
                print('md: '+root+'/'+file)
                with open(os.path.join(root, file), encoding='utf-8', errors="ignore") as fp:
                    content = fp.read()
                matches = text_match(content)
                if bool(matches):
                    file_meta['keywords'] = matches
                
            if bool(file_meta):
                relative = os.path.relpath(root, dir)
                file = relative+'\\'+file
                print('relative: '+file)
                
                #if 'libs' in file_meta:
                #    file_meta['ml_libs'] = list(package_match(file_meta['libs']))
                
                meta.append({'file': file, 'meta': file_meta})
                #print(file_meta)

    if store:
        path_meta = os.path.join(dir+path_meta)
        print('meta: '+path_meta)
        #create_folder(path_meta)
        with open(path_meta, 'w') as fp:
            fp.write(json.dumps(meta))
    
    return meta
                
meta = dir_walk(dir_base+dir6)
print(meta)
#print(json.dumps(meta))

md: ../data/repositories/git/firmai\financial-machine-learning\/README.md
relative: .\README.md
meta: ../data/repositories/git/firmai\financial-machine-learning\/.meta\git_analyzer_results_v02.json
[{'file': '.\\README.md', 'meta': {'keywords': ['AI', 'DL', 'ML', 'NLP', 'NN', 'RL', 'autoencoder', 'bayesian', 'classif', 'clustering', 'deep learning', 'detect', 'gru', 'layer', 'lstm', 'machine learning', 'model', 'neural network', 'predict', 'recurrent neural network', 'regression', 'reinforcement learning', 'sentiment analysis']}}]


In [22]:
#dir_base = '../data/repositories/git/'
dir_base = '../data/repositories/blobcity/git/'

authors = os.listdir(dir_base)

a = 0
p = 0

start = 0
end = 2000

runtime_start = time.time()

i = start
for author in authors:
    if os.path.isdir(os.path.join(dir_base, author)):
        
        projects = os.listdir(os.path.join(dir_base, author))
        #print(author, projects)
        for project in projects:
            #print(project)
            if p >= start:
                if os.path.isdir(os.path.join(dir_base, author, project)):
                    print('### ' + str(i) + ': ' + author, project)
                    meta = dir_walk(os.path.join(dir_base, author, project))
                    #print(meta)
                    
                i += 1

            # count projects
            p += 1
        #count authors
        a += 1
        
    # break loop
    if i>=end and True:
        break
    
runtime_end = time.time()
print('### parsed {0} git-projects  in {1} seconds ###'.format(i-start, round(runtime_end - runtime_start, 3)))

### 1000: z1ming LeetCode-Notebook
md: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook/README.md
relative: .\README.md
md: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook/README_en.md
relative: .\README_en.md
md: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook/如何使用 Github 进行多人协作.md
relative: .\如何使用 Github 进行多人协作.md
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0001.两数之和/0001-两数之和.py
relative: source\All_Solutions\0001.两数之和\0001-两数之和.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0002.两数相加/0002-两数相加.py
relative: source\All_Solutions\0002.两数相加\0002-两数相加.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0003.无重复字符的最长子串/0003-无重复字符的最长子串.py
relative: source\All_Solutions\0003.无重复字符的最长子串\0003-无重复字符的最长子串.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0004.寻找两个有序数组的中位数/0004-寻找两个有序数组的中位数.py
relative: source\All

py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0503.下一个更大元素II/0503-下一个更大元素II.py
relative: source\All_Solutions\0503.下一个更大元素II\0503-下一个更大元素II.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0505.迷宫II/0505-迷宫II.py
relative: source\All_Solutions\0505.迷宫II\0505-迷宫II.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0508.出现次数最多的子树元素和/0508-出现次数最多的子树元素和.py
relative: source\All_Solutions\0508.出现次数最多的子树元素和\0508-出现次数最多的子树元素和.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0510.二叉搜索树中的中序后继II/0510-二叉搜索树中的中序后继II.py
relative: source\All_Solutions\0510.二叉搜索树中的中序后继II\0510-二叉搜索树中的中序后继II.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0513.找树左下角的值/0513-找树左下角的值.py
relative: source\All_Solutions\0513.找树左下角的值\0513-找树左下角的值.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0514.自由之路/0

relative: source\All_Solutions\0721.账户合并\0721-账户合并.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0722.删除注释/0722-删除注释.py
relative: source\All_Solutions\0722.删除注释\0722-删除注释.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0723.粉碎糖果/0723-粉碎糖果.py
relative: source\All_Solutions\0723.粉碎糖果\0723-粉碎糖果.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0725.分隔链表/0725-分隔链表.py
relative: source\All_Solutions\0725.分隔链表\0725-分隔链表.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0765.情侣牵手/0765-情侣牵手.py
relative: source\All_Solutions\0765.情侣牵手\0765-情侣牵手.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0766.托普利茨矩阵/0766-托普利茨矩阵.py
relative: source\All_Solutions\0766.托普利茨矩阵\0766-托普利茨矩阵.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0767.重构字符串/0767-重构字符串.py
relative: source\All_Sol

relative: source\All_Solutions\0975.奇偶跳\0975-奇偶跳.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0978.最长湍流子数组/0978-最长湍流子数组.py
relative: source\All_Solutions\0978.最长湍流子数组\0978-最长湍流子数组.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0979.在二叉树中分配硬币/0979-在二叉树中分配硬币.py
relative: source\All_Solutions\0979.在二叉树中分配硬币\0979-在二叉树中分配硬币.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0980.不同路径III/0980-不同路径III.py
relative: source\All_Solutions\0980.不同路径III\0980-不同路径III.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0981.基于时间的键值存储/0981-基于时间的键值存储.py
relative: source\All_Solutions\0981.基于时间的键值存储\0981-基于时间的键值存储.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\0982.按位与为零的三元组/0982-按位与为零的三元组.py
relative: source\All_Solutions\0982.按位与为零的三元组\0982-按位与为零的三元组.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Noteb

py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\1178.猜字谜/1178-猜字谜.py
relative: source\All_Solutions\1178.猜字谜\1178-猜字谜.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\1180.统计只含单一字母的子串/1180-统计只含单一字母的子串.py
relative: source\All_Solutions\1180.统计只含单一字母的子串\1180-统计只含单一字母的子串.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\1181.前后拼接/1181-前后拼接.py
relative: source\All_Solutions\1181.前后拼接\1181-前后拼接.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\1182.与目标颜色间的最短距离/1182-与目标颜色间的最短距离.py
relative: source\All_Solutions\1182.与目标颜色间的最短距离\1182-与目标颜色间的最短距离.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\1184.公交站间的距离/1184-公交站间的距离.py
relative: source\All_Solutions\1184.公交站间的距离\1184-公交站间的距离.py
py: ../data/repositories/blobcity/git/z1ming\LeetCode-Notebook\source\All_Solutions\1186.删除一次得到子数组最大和/1186-删除一次得到子数组最大和.py
relative

...

relative: learn-go\gin版g站\giligili\README.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\gin版g站\giligili\project-images/readme.md
relative: learn-go\gin版g站\giligili\project-images\readme.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\gin版g站\giligili\swagger/README.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\gin版g站\giligili-vue/README.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\golang读书笔记/01-入门.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\golang读书笔记/01-数据类型.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\golang读书笔记/02-变量.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\golang读书笔记/03-常量.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\golang读书笔记/04-作用域.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\learn-go\golang读书笔记/05-约定和惯例.md
md: ../data/repositories/blobcity/

md: ../data/repositories/blobcity/git/zhang14725804\notebook\react-vue\react项目\react-chat/README.md
relative: react-vue\react项目\react-chat\README.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\react-vue\react项目\react-chat\server\node_modules\_accepts@1.3.5@accepts/HISTORY.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\react-vue\react项目\react-chat\server\node_modules\_accepts@1.3.5@accepts/README.md
relative: react-vue\react项目\react-chat\server\node_modules\_accepts@1.3.5@accepts\README.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\react-vue\react项目\react-chat\server\node_modules\_accepts@1.3.5@accepts\node_modules\mime-types/HISTORY.md
md: ../data/repositories/blobcity/git/zhang14725804\notebook\react-vue\react项目\react-chat\server\node_modules\_accepts@1.3.5@accepts\node_modules\mime-types/README.md
relative: react-vue\react项目\react-chat\server\node_modules\_accepts@1.3.5@accepts\node_modules\mime-types\README.md
md: ../data/repositor

md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/C-bigLine.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/CWebServer-CGI.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/easyDeployEvolution.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/goDesignPattern1.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/goDesignPattern2.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/goDesignPattern3.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/gotorch.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/jsAlertOrder.md
relative: blog\jsAlertOrder.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/keywordsMatchOptimization.md
relative: blog\keywordsMatchOptimization.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/linuxBackEndJob.md
md: ../data/repositories/blobcity/git/zhenbianshu\notebook\blog/linuxMonitor.md
md: ../data/repo

relative: .\Linux-setup.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/Linux-SSH.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/Linux-VirtualBox.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/Misson.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/MySQL.md
relative: .\MySQL.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/Nginx.md
relative: .\Nginx.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/PaperReading.md
relative: .\PaperReading.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/PHP.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/Python.md
relative: .\Python.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/PythonCourse.md
relative: .\PythonCourse.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/RabbitMQ.md
relative: .\RabbitMQ.md
md: ../data/repositories/blobcity/git/zjuchenyuan\notebook/README.md
relative: .\README.md
md: ../data/repositories/b

relative: .\FMM.ipynb
ipynb: ../data/repositories/blobcity/git/znah\notebooks/geodesic.ipynb
relative: .\geodesic.ipynb
ipynb: ../data/repositories/blobcity/git/znah\notebooks/guided.ipynb
relative: .\guided.ipynb
py: ../data/repositories/blobcity/git/znah\notebooks/guided.pyx
relative: .\guided.pyx
py: ../data/repositories/blobcity/git/znah\notebooks/hilbert.py
relative: .\hilbert.py
ipynb: ../data/repositories/blobcity/git/znah\notebooks/hilbert_curve.ipynb
relative: .\hilbert_curve.ipynb
ipynb: ../data/repositories/blobcity/git/znah\notebooks/mini_sinkhorn.ipynb
relative: .\mini_sinkhorn.ipynb
py: ../data/repositories/blobcity/git/znah\notebooks/nbutils.py
relative: .\nbutils.py
py: ../data/repositories/blobcity/git/znah\notebooks/ply.py
relative: .\ply.py
md: ../data/repositories/blobcity/git/znah\notebooks/README.md
py: ../data/repositories/blobcity/git/znah\notebooks/sgm.pyx
relative: .\sgm.pyx
ipynb: ../data/repositories/blobcity/git/znah\notebooks/stereo.ipynb
relative: .\stere